In [ ]:
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2
from sklearn.svm import SVC,SVR
from src.MFTreeSearchCV import *
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb

In [ ]:
from sklearn.datasets import load_digits,load_boston,fetch_20newsgroups
data = load_boston()

In [ ]:
newsgroups_train = fetch_20newsgroups(subset='all')
vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform(newsgroups_train.data)
labels = newsgroups_train.target
#features =features.todense()

In [ ]:
X = features
y = labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape

In [ ]:
estimator = xgb.XGBClassifier(max_depth=3,n_estimators=100,colsample_bytree=0.8,nthread=4)
param_dict = {'max_depth':{'range':[2,6],'scale':'linear','type':'int'},
'n_estimators':{'range':[50,300],'scale':'linear','type':'int'},\
'colsample_bytree':{'range':[0.3,0.9],'scale':'linear','type':'real'},
'learning_rate':{'range':[0.03,0.1],'scale':'log','type':'real'}}
fidelity_range = [500,15076]
n_jobs = 2
cv = 5
fixed_params = {}
scoring = 'accuracy'
              

In [ ]:
t1 = time.time()
#estimator = estimator.fit(X_train,y_train)
t2 = time.time()
unit_cost = cv*(t2 - t1)
total_budget = 2000

In [ ]:
model = MFTreeSearchCV(estimator=estimator,param_dict=param_dict,scoring=scoring,\
                      fidelity_range=fidelity_range,unit_cost=None,\
                    cv=5,  n_jobs = n_jobs,total_budget=total_budget,debug = False,fixed_params=fixed_params)

In [ ]:
m = model.fit(X_train,y_train)

In [ ]:
m.best_params_

In [ ]:
y_pred = m.predict(X_test)


# newm = xgb.XGBRegressor()
# newm = newm.set_params(**m.best_params_)
# newm = newm.fit(X_train,y_train)
# y_pred = newm.predict(X_test)

In [ ]:
r2_score(y_pred,y_test)

In [ ]:
m.MP.unit_cost

In [ ]:
m.best_params_

In [ ]:
m.points
#m.evals

In [ ]:
t2 -t1